os.path.split 的使用方法

In [ ]:
import os

path = 'C:/Users/xxx/Desktop/新建文件夹/12/aa123.pdf'
head_tail = os.path.split(path)

print("Head of '% s:'" % path, head_tail[0]) 
print("Tail of '% s:'" % path, head_tail[1], "\n") 
print(type(head_tail))

使用 os.path.splitext 进一步分离地址下的文件名与后缀名

In [ ]:
import os

path = 'C:/Users/xxx/Desktop/新建文件夹/12/ab123.pdf'
head_tail = os.path.split(path)

file_type = os.path.splitext(head_tail[1])
print("Head of '% s:'" % path, file_type[0]) 
print("Tail of '% s:'" % path, file_type[1], "\n") 
print(type(file_type))

使用正则判断文件名中是否包含数字

In [ ]:
import os, re

path = 'C:/Users/xxx/Desktop/新建文件夹/12/ab123.pdf'
head_tail = os.path.split(path)

pattern = re.compile('[0-9]+')
match = pattern.findall(head_tail[1])
if match:
    print ('contains digital')
else:
    print ('not contains digital')

使用 isdigit 判断字符串是否以数字开头

In [ ]:
import os

path = 'C:/Users/xxx/Desktop/新建文件夹/12/ab123.pdf'
head_tail = os.path.split(path)

if head_tail[1].isdigit():
    print('start with digit')
else:
    print('not start with digit')

将文件地址拆分并获取文件夹名称

In [ ]:
import os

path = 'C:/Users/xxx/Desktop/新建文件夹/12/ab123.pdf'
head_tail = os.path.split(path)

head_tail = os.path.split(path)
file_dir = head_tail[0].split('/')
print(file_dir)
print(file_dir[-1])

修改文件名

In [ ]:
import os

path = "C:/Users/巴图/Desktop/新建文件夹/高梓函/20220508_1.jpg"
head_tail = os.path.split(path)
if head_tail[1].endswith('.jpg'):   #当文件名以.txt后缀结尾时
    old_name = head_tail[1]
    new_name=old_name.replace('.jpg','_1.jpg')   #将原来名字里的‘.txt’替换为‘-test.txt’,相当于加个后缀了
    os.rename(os.path.join(head_tail[0],old_name),os.path.join(head_tail[0],new_name))

文件是否存在

In [ ]:
import os
os.path.exists("C:/Users/巴图/Desktop/新建文件夹/高梓函/20220508_1.jpg")

使用openpyxl获取xlsx中的数据

In [ ]:
from openpyxl import load_workbook
wb = load_workbook('26.Scratch编程课程信息汇总.xlsx')
ws = wb.active

row_max = ws.max_row
column_max = ws.max_column

print(row_max)
print(column_max)

data = []
for row in ws.iter_rows(min_row=2, max_row=35, min_col=5, max_col=8):
    values = []
    for cell in row:
        values.append(cell.value)
    data.append(values)
# print(data)

def remove_none_in_list(ori_list):
    new_list = []
    for item in ori_list:
        if item != None:
            new_list.append(item)
    return new_list

# remove_none_in_list(data)

a = tuple(ws.iter_rows(min_row=2, max_row=35, min_col=5, max_col=8))

for x in [0,2,3]:
    b = a[0][x].value
    print()
    print(b)


使用ics从iCalendar中获取日历信息

In [ ]:
from ics import Calendar
from dateutil.relativedelta import relativedelta
import requests, datetime
url = 'https://outlook.live.com/owa/calendar/00000000-0000-0000-0000-000000000000/1ae29d99-e0f2-4b2b-982f-1a6c2ebf884f/cid-F0D08E876E592DC1/calendar.ics'
c = Calendar(requests.get(url).text)

start_date = datetime.date(2022,5,1)
end_date = start_date + relativedelta(months=1)

e = list(c.timeline)

for calender_event in e:
    if calender_event.begin.date() > start_date and calender_event.begin.date() < end_date:
        print("Event '{}' started {}".format(calender_event.name, calender_event.begin.format('YYYY-MM-DD HH:MM')))
        # print(type(calender_event.begin.date()))


使用requests_testadapter创建本地请求，代替requests，获取本地文件

In [ ]:
from requests_testadapter import Resp
class LocalFileAdapter(requests.adapters.HTTPAdapter):
    def build_response_from_file(self, request):
        file_path = request.url[7:]
        with open(file_path, 'rb') as file:
            buff = bytearray(os.path.getsize(file_path))
            file.readinto(buff)
            resp = Resp(buff)
            r = self.build_response(request, resp)

            return r

    def send(self, request, stream=False, timeout=None,
             verify=True, cert=None, proxies=None):

        return self.build_response_from_file(request)

requests_session = requests.session()
requests_session.mount('file://', LocalFileAdapter())
r = requests_session.get('file://图图的课表.ics')

从ics中获取上课记录并保存在list中

In [ ]:
# 从ics中获取上课记录并保存在list中

from ics import Calendar
from dateutil.relativedelta import relativedelta
import requests, datetime
url = 'https://outlook.live.com/owa/calendar/3d1bd554-c7f0-49e2-996b-2cf4a6b39fa7/685e27f6-3554-4af1-b07c-4d785a347cc5/cid-28689F54F556F71F/calendar.ics'
c = Calendar(requests.get(url).text)
# c = Calendar(r.text)  # 使用本地ics文件访问

now = datetime.datetime.now()
year = now.year
month = now.month
start_date = datetime.date(2022,month,1)
end_date = start_date + relativedelta(months=1)

e = list(c.timeline)    # 日历中的全部事件

class_name = [] # 获取全部的课程名称
for calender_event in e:    # 获取选定日期的全部课程名称，并添加到列表中
    if (calender_event.begin.date() > start_date) and (calender_event.begin.date() < end_date): # 筛选指定日期内的事件
        # print(calender_event.name)
        item = calender_event.name.split(' ')[0]    # 获取课程名称
        if item not in class_name:
            class_name.append(item)

class_in_total = []
for calender_event in e:
    current_class = []
    if calender_event.begin.date() > start_date and calender_event.begin.date() < end_date: # 筛选指定日期内的事件
        try:
            c1ass = calender_event.name.split(' ')[0]
            student = len(calender_event.name.split(' ')[1:])
            class_date = calender_event.begin.format('YYYY-MM-DD HH:mm').split(' ')[0]
            class_time = calender_event.begin.format('YYYY-MM-DD HH:mm').split(' ')[1]
            current_class.append(c1ass)
            current_class.append(class_date)
            current_class.append(class_time)
            current_class.append(student)
            # print(current_class)
            # print(c1ass)   # 课程名称
            # print(student)   # 学生名称
            # print(calender_event.begin.format('YYYY-MM-DD HH:mm:ss'))   # 课程时间
        except IndexError:
            pass
            # print(calender_event.name.split(' ')[0])   # 课程名称
            # print(calender_event.begin.format('YYYY-MM-DD HH:mm:ss'))   # 课程时间

        class_in_total.append(current_class)
class_in_total.sort()
# print(class_in_total)
print('日历中共有'+str(len(class_in_total))+'项日程')

将列表中的课程导入至课时统计表中

In [26]:
# 将列表中的课程导入至课时统计表中

from openpyxl import load_workbook
import os

wb = load_workbook(filename = '教师课时统计表模板.xlsx')
ws = wb.active

cols = {'启蒙':2, '玛塔':5, '程小奔':5, '探究':8, '工程':11, 'Scratch':14, 'Python':14, '南开':17, '普林斯顿':17, '博苑澳森':17}

def insert_course_information(calender_list_item, course):  # 
    time_value = datetime.datetime.strptime(calender_list_item[1],'%Y-%m-%d')
    time_value = time_value.strftime('%m月%d日')
    if calender_list_item[0] == course:
        row = 5
        col = cols[course]
        while ws.cell(row=row, column=col).value != None :
            # print(str(row) + ' ' +str(col) + ' is not empty')
            row += 1
            if ws.cell(row=row, column=col).value == None:
                break
        if course in ('南开','普林斯顿','博苑澳森'):    # 课程格式不同
            ws.cell(row=row, column=col, value=course)
            ws.cell(row=row, column=col+1, value=time_value)
            ws.cell(row=row, column=col+2, value=calender_list_item[2])
            ws.cell(row=row, column=col+3, value=calender_list_item[3])
            row += 1
            wb.save(filename='教师课时统计表模板.xlsx')
        else:
            ws.cell(row=row, column=col, value=time_value)
            ws.cell(row=row, column=col+1, value=calender_list_item[2])
            ws.cell(row=row, column=col+2, value=calender_list_item[3])
            row += 1
            wb.save(filename='教师课时统计表模板.xlsx')

for item in class_in_total:
    insert_course_information(item, item[0])

new_file_name = '图图'+str(year)+'年'+str(month)+'月课时统计.xlsx'
os.rename('教师课时统计表模板.xlsx', new_file_name)